In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索ツールの定義
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
tools = [tool]

In [3]:
tool.invoke("Langgraphのノードとは？")

[{'url': 'https://weel.co.jp/media/tech/langgraph/',
  'content': '参考：https://github.com/langchain-ai/langgraph/blob/main/docs/docs/how-tos/visualization.ipynb\nLangGraphを使うことにより複数のエージェントを定義することができ、それぞれに特定の役割を持たせることができます。また、各ステップを「ノード」として定義して「状態の遷移」を制御可能。さらに、条件付き遷移や処理の再実行が簡単にできるなどのメリットがあります。\nLangGraphはRAGのような使い方もできますが、LangGraphとRAGは異なっており、RAGはLLMに不足している知識を外部データベースから補完するのに対して、LangGraphはその処理の流れ(検索→生成→出力など)をノードとして組み、順序や条件を制御する役割を担っています。\nLangGraphの構成要素\nLangGraphにはノードや状態の遷移以外にも構成要素があります。※1\n先ほどの画像と見比べながら解説をします。 [...] ホーム\n生成AIテック\nやってみた系\nLangGraphとは？エージェントの使い方とワークフロー構築法をハンズオンで解説\n\nLangGraphとは？エージェントの使い方とワークフロー構築法をハンズオンで解説\n2025 4/10\nやってみた系\n2025-04-10\n\n押さえておきたいポイント\n\nLangChainのコンポーネントを使って処理フローをグラフ構造で制御できる\nノード間で状態（データ）を引き継ぎながら処理を進められる\nローカル環境で実装可能\n\nLangChainに似たLangGraphをご存知ですか？LangGraphはReplitやUber、LinkedInなどで使用されており、LangChainでは複雑で対応が難しかった処理も制御可能です。\n本記事では、LangGraphの概要やLangChainとの違い、実際の使い方について解説をします。最後までお読みいただければ、LangGraphの理解が深まり、実装できるようになります。\n\n目次\n\nLangGraphの概要\nLangGraphの構成要

In [4]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

# Stateクラスの定義
class State(TypedDict):
    messages: Annotated[list, add_messages]

# グラフのインスタンスを作成
graph_builder = StateGraph(State)

# 言語モデルの定義
llm = ChatOpenAI(model_name=MODEL_NAME)

# 変更点：ツール定義の紐づけ
llm_with_tools = llm.bind_tools(tools)

# チャットボットノードの作成
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# グラフにチャットボットノードを追加
graph_builder.add_node("chatbot", chatbot)

In [5]:
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ツールノードの作成
tool_node = ToolNode(tools)

# グラフにツールノードを追加
graph_builder.add_node("tools", tool_node)

# 条件付エッジの作成
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition, # ツール呼出と判断したらツールノードを呼ぶ
)

# ツールが呼び出されるたびに、チャットボットに戻って次のステップを決定
# ツールからチャットボットへの戻りエッジを作成
graph_builder.add_edge("tools", "chatbot")

# 開始ノードの指定
graph_builder.set_entry_point("chatbot")

# 記憶を持つ実行可能なステートグラフの作成
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [6]:
# グラフの実行と結果の表示
def stream_graph_updates(user_input: str):
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# チャットボットのループ
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(user_input)

こんにちは！
こんにちは！今日はどのようなお手伝いができますか？
1足す2は？
1足す2は3です。何か他にお手伝いできることがありますか？
1メートル以上の魚は？

[{"url": "http://cms.town.wakayama-hidaka.lg.jp/docs/2014090300054/", "content": "少々いかつい顔をしたこの“どでかい魚”、名前はクエ。 大きいものでは、体長1メートル以上にも達する大型魚です。関東以南の太平洋沿岸から東シナ海沿岸にかけての温かい海"}, {"url": "https://detail.chiebukuro.yahoo.co.jp/qa/question_detail/q14260099909", "content": "Yahoo!知恵袋\n\nJavaScriptが無効です。ブラウザの設定でJavaScriptを有効にしてください。JavaScriptを有効にするには\n\n回答受付が終了しました\n\nzka********さん\n\n2022/4/10 10:34\n\n44回答\n\n友人と琵琶湖でバス釣りをしていたら、余裕で1m以上の巨大な魚を見ました。かなり近くで見たのですが、ビワコオオナマズでも、コイでもありません。体の長さに対して太さはさほどありませんでした。\n\n友人と琵琶湖でバス釣りをしていたら、余裕で1m以上の巨大な魚を見ました。かなり近くで見たのですが、ビワコオオナマズでも、コイでもありません。体の長さに対して太さはさほどありませんでした。\n肌色に近い体色をしていました。おそらくソウギョかなぁと思いましたが、琵琶湖でソウギョ見たことある方はいますか？\r\nまた、他の魚の可能性があるとしたら、何でしょうか？\n\n釣り・1,118閲覧\n\n1人が共感しています\n\n回答（4件）\n\nwedssport205さん\n\n2022/4/11 22:46 [...] 南湖東岸エリアか天神川付近ならアオウオですね。特に水面付近でヒレ立てて泳いでりゃ間違いなくアオウオ。\n\nこの回答はいかがでしたか？ リアクションしてみよう\n\nなるほど\n\n0\n\nそうだね\n\n0\n\nありがとう\n\n0\n\nzka********さん\n\n質問者2022/4/13 14:54\